In [1]:
import _init_paths
from fast_rcnn.train_mv import get_training_roidb, train_net
from fast_rcnn.config import cfg,cfg_from_file, cfg_from_list, get_output_dir
from datasets.factory import get_imdb
from networks.factory import get_network
import argparse
import pprint
import numpy as np
import sys
import pdb

- __name : kitti_train
- name : <type 'function'>
- __name : kitti_val
- name : <type 'function'>
- __name : kitti_trainval
- name : <type 'function'>
- __name : kitti_test
- name : <type 'function'>


In [2]:
#!python ./train_net.py --device gpu --device_id 0 --weights /workspace/MV3D_TF/data/pretrain_model/VGG_imagenet.npy --iters 1 --cfg /workspace/MV3D_TF/experiments/cfgs/faster_rcnn_end2end.yml --network MV3D_train

# 1. get_imdb()

In [3]:
imdb_name = 'kitti_train'
imdb = get_imdb(imdb_name)

- get_imdb(name) : kitti_train
<bound method kitti_mv3d.default_roidb of <datasets.kitti_mv3d.kitti_mv3d object at 0x7f72ad76df90>>
-$$$$ PATH : ../lib/datasets/../../data/KITTI---train
image sets length:  4682
- image_set_file : ../lib/datasets/../../data/KITTI/ImageSets/train.txt


```python
def get_imdb(name):
    """Get an imdb (image database) by name."""
    print("- get_imdb(name) : {}".format(name))
    if not __sets.has_key(name):
        raise KeyError('Unknown dataset: {}'.format(name))

    return __sets[name]()
```
## 1.1 sets[name]처리 부분
```
for split in ['train', 'val', 'trainval', 'test']:
    name = 'kitti_{}'.format(split)
    print("- __name : {}".format(name))
    __sets[name] = (lambda split=split:
            datasets.kitti_mv3d.kitti_mv3d(split))
```

## 1.2 datasets.kitti_mv3d.kitti_mv3d(split) 가 실행 : 경로 및 확장자 설정

```python
class kitti_mv3d(datasets.imdb):
    def __init__(self, image_set, kitti_path=None):
        datasets.imdb.__init__(self, image_set)
        self._image_set = image_set
        self._kitti_path = self._get_default_path() if kitti_path is None \
                            else kitti_path

        self._data_path = os.path.join(self._kitti_path, 'object')
        self._classes = ('__background__', 'Car')#, 'Pedestrian', 'Cyclist')
        self._class_to_ind = dict(zip(self.classes, xrange(self.num_classes)))
        self._image_ext = '.png'
        self._lidar_ext = '.npy'
        self._subset = 'car'
        self._image_index = self._load_image_set_index() #IamgeSets/train.txt파일, 4681개의 숫자 list
        self._roidb_handler = self.gt_roidb

        self.config = {'top_k': 100000}
        assert os.path.exists(self._kitti_path), \
                'KITTI path does not exist: {}'.format(self._kitti_path)
        assert os.path.exists(self._data_path), \
                'Path does not exist: {}'.format(self._data_path)
```

## 1.3 datasets.imdb.__init__(self, image_set)
```
    def __init__(self, name):
        self._name = name
        self._num_classes = 0
        self._classes = []
        self._image_index = []
        self._obj_proposer = 'selective_search'
        self._roidb = None
        print self.default_roidb
        self._roidb_handler = self.default_roidb
        # Use this dict for storing dataset specific config options
        self.config = {}
 ```


In [4]:
print 'Loaded dataset `{:s}` for training'.format(imdb.name)

Loaded dataset `train` for training


In [5]:
print("-imdb.classes:{}".format(imdb.classes))
print("-imdb._num_classes:{}".format(imdb._num_classes))
#print("-imdb._classes:{}".format(imdb._classes))
#print("-imdb._image_index:{}".format(imdb._image_index))
print("-imdb._obj_proposer:{}".format(imdb._obj_proposer))
print("-type(imdb._roidb):{}".format(type(imdb._roidb)))

-imdb.classes:('__background__', 'Car')
-imdb._num_classes:0
-imdb._obj_proposer:selective_search
-type(imdb._roidb):<type 'NoneType'>


# 2. roidb = get_training_roidb(imdb)

In [6]:
roidb = get_training_roidb(imdb)
type(roidb)

- get_training_roidb(imdb) : <datasets.kitti_mv3d.kitti_mv3d object at 0x7f72ad76df90>
Preparing training data...
call : rdl_roidb.prepare_roidb(imdb)
train gt roidb loaded from /workspace/MV3D_TF/data/cache/train_gt_roidb.pkl
done


list

```python
def get_training_roidb(imdb):
    """Returns a roidb (Region of Interest database) for use in training."""
    print("- get_training_roidb(imdb) : {}".format(imdb))
    if cfg.TRAIN.USE_FLIPPED:
        print 'Appending horizontally-flipped training examples...'
        imdb.append_flipped_images()
        print 'done'

    print 'Preparing training data...'
    if cfg.TRAIN.HAS_RPN:
        if cfg.IS_MULTISCALE:
            gdl_roidb.prepare_roidb(imdb)
        else:
            rdl_roidb.prepare_roidb(imdb)
    else:
        rdl_roidb.prepare_roidb(imdb)  ## called 
    print 'done'

    return imdb.roidb
```

## 2.1  rdl_roidb.prepare_roidb(imdb)

```python
# import roi_data_layer.roidb as rdl_roidb

def prepare_roidb(imdb):
    """Enrich the imdb's roidb by adding some derived quantities that
    are useful for training. This function precomputes the maximum
    overlap, taken over ground-truth boxes, between each ROI and
    each ground-truth box. The class with maximum overlap is also
    recorded.
    """
    roidb = imdb.roidb

    for i in xrange(len(imdb.image_index)):

        if roidb[i]['boxes_corners'] == []:
            print 'boxes_corners not correct', imdb.image_path_at(i)
            continue
        roidb[i]['image_path'] = imdb.image_path_at(i)
        roidb[i]['lidar_bv_path'] = imdb.lidar_path_at(i)
        roidb[i]['calib'] = imdb.calib_at(i)

        # need gt_overlaps as a dense array for argmax
        gt_overlaps = roidb[i]['gt_overlaps'].toarray()
        max_overlaps = gt_overlaps.max(axis=1)
        max_classes = gt_overlaps.argmax(axis=1)

        roidb[i]['max_classes'] = max_classes
        roidb[i]['max_overlaps'] = max_overlaps
        zero_inds = np.where(max_overlaps == 0)[0]
        nonzero_inds = np.where(max_overlaps > 0)[0]


```

In [12]:
output_dir = get_output_dir(imdb, None)
print 'Output will be saved to `{:s}`'.format(output_dir)

Output will be saved to `/workspace/MV3D_TF/output/default/train`


# 3.     network = get_network(args.network_name)

In [7]:
network_name = "MV3D_train"
network = get_network(network_name)

Tensor("Placeholder:0", shape=(?, ?, ?, 9), dtype=float32)
Tensor("Placeholder_1:0", shape=(?, ?, ?, 3), dtype=float32)
Tensor("conv5_3/conv5_3:0", shape=(?, ?, ?, 512), dtype=float32)
Tensor("rpn_cls_score/rpn_cls_score:0", shape=(?, ?, ?, 8), dtype=float32)
Tensor("Placeholder_4:0", shape=(?, 5), dtype=float32)
Tensor("Placeholder_5:0", shape=(?, 7), dtype=float32)
Tensor("Placeholder_2:0", shape=(?, 3), dtype=float32)
Tensor("rpn_conv/3x3/rpn_conv/3x3:0", shape=(?, ?, ?, 512), dtype=float32)
Tensor("rpn_cls_score/rpn_cls_score:0", shape=(?, ?, ?, 8), dtype=float32)
Tensor("rpn_cls_prob:0", shape=(?, ?, ?, ?), dtype=float32)
Tensor("Reshape_2:0", shape=(?, ?, ?, 8), dtype=float32)
Tensor("rpn_bbox_pred/rpn_bbox_pred:0", shape=(?, ?, ?, 24), dtype=float32)
Tensor("Placeholder_2:0", shape=(?, 3), dtype=float32)
Tensor("Placeholder_7:0", shape=(?, 12), dtype=float32)
(<tf.Tensor 'rpn_rois/rpn_rois_bv:0' shape=(?, 5) dtype=float32>, <tf.Tensor 'rpn_rois/rpn_rois_img:0' shape=(?, 5) dtype

```python
def get_network(name):
    """Get a network by name."""
    #if not __sets.has_key(name):
    #    raise KeyError('Unknown dataset: {}'.format(name))
    #return __sets[name]
    if name.split('_')[1] == 'test':
       return MV3D_test()
    elif name.split('_')[1] == 'train':
       return MV3D_train()
    else:
       raise KeyError('Unknown dataset: {}'.format(name))
```
## 3.1  MV3D_train()
```python
class MV3D_train(Network):
    def __init__(self, trainable=True):
        self.inputs = []
        self.lidar_bv_data = tf.placeholder(tf.float32, shape=[None, None, None, 9])
        self.image_data = tf.placeholder(tf.float32, shape=[None, None, None, 3])
        self.im_info = tf.placeholder(tf.float32, shape=[None, 3])
        self.gt_boxes = tf.placeholder(tf.float32, shape=[None, 5])
        self.gt_boxes_bv = tf.placeholder(tf.float32, shape=[None, 5])
        self.gt_boxes_3d = tf.placeholder(tf.float32, shape=[None, 7])
        self.gt_boxes_corners = tf.placeholder(tf.float32, shape=[None, 25])
        self.calib = tf.placeholder(tf.float32, shape=[None, 12])
        self.keep_prob = tf.placeholder(tf.float32)
        self.layers = dict({'lidar_bv_data':self.lidar_bv_data,
                            'image_data':self.image_data,
                            'calib' : self.calib,
                            'im_info':self.im_info,
                            'gt_boxes':self.gt_boxes,
                            'gt_boxes_bv':self.gt_boxes_bv,
                            'gt_boxes_3d': self.gt_boxes_3d,
                            'gt_boxes_corners': self.gt_boxes_corners})
        self.trainable = trainable
        self.setup()

        # create ops and placeholders for bbox normalization process
        with tf.variable_scope('bbox_pred', reuse=True):
            weights = tf.get_variable("weights")
            biases = tf.get_variable("biases")

            self.bbox_weights = tf.placeholder(weights.dtype, shape=weights.get_shape())
            self.bbox_biases = tf.placeholder(biases.dtype, shape=biases.get_shape())

            self.bbox_weights_assign = weights.assign(self.bbox_weights)
            self.bbox_bias_assign = biases.assign(self.bbox_biases)

    def setup(self):
        # Lidar Bird View
        (self.feed('lidar_bv_data')
             .conv(3, 3, 64, 1, 1, name='conv1_1')
             .conv(3, 3, 64, 1, 1, name='conv1_2')
             .max_pool(2, 2, 2, 2, padding='VALID', name='pool1')
             .conv(3, 3, 128, 1, 1, name='conv2_1')
             .conv(3, 3, 128, 1, 1, name='conv2_2')
             .max_pool(2, 2, 2, 2, padding='VALID', name='pool2')
             .conv(3, 3, 256, 1, 1, name='conv3_1')
             .conv(3, 3, 256, 1, 1, name='conv3_2')
             .conv(3, 3, 256, 1, 1, name='conv3_3')
             .max_pool(2, 2, 2, 2, padding='VALID', name='pool3')
             .conv(3, 3, 512, 1, 1, name='conv4_1')
             .conv(3, 3, 512, 1, 1, name='conv4_2')
             .conv(3, 3, 512, 1, 1, name='conv4_3')
             .conv(3, 3, 512, 1, 1, name='conv5_1')
             .conv(3, 3, 512, 1, 1, name='conv5_2')
             .conv(3, 3, 512, 1, 1, name='conv5_3'))
        # RGB
        (self.feed('image_data')
              .conv(3, 3, 64, 1, 1, name='conv1_1_2')
              .conv(3, 3, 64, 1, 1, name='conv1_2_2')
              .max_pool(2, 2, 2, 2, padding='VALID', name='pool1_2')
              .conv(3, 3, 128, 1, 1, name='conv2_1_2')
              .conv(3, 3, 128, 1, 1, name='conv2_2_2')
              .max_pool(2, 2, 2, 2, padding='VALID', name='pool2_2')
              .conv(3, 3, 256, 1, 1, name='conv3_1_2')
              .conv(3, 3, 256, 1, 1, name='conv3_2_2')
              .conv(3, 3, 256, 1, 1, name='conv3_3_2')
              .max_pool(2, 2, 2, 2, padding='VALID', name='pool3_2')
              .conv(3, 3, 512, 1, 1, name='conv4_1_2')
              .conv(3, 3, 512, 1, 1, name='conv4_2_2')
              .conv(3, 3, 512, 1, 1, name='conv4_3_2')
              .conv(3, 3, 512, 1, 1, name='conv5_1_2')
              .conv(3, 3, 512, 1, 1, name='conv5_2_2')
              .conv(3, 3, 512, 1, 1, name='conv5_3_2'))

        #========= RPN ============

        (self.feed('conv5_3')
             # .deconv(shape=None, c_o=512, stride=2, ksize=3,  name='deconv_2x_1')
             .conv(3,3,512,1,1,name='rpn_conv/3x3')
             .conv(1,1,len(anchor_scales)*2*2 ,1 , 1, padding='VALID', relu = False, name='rpn_cls_score'))

        (self.feed('rpn_cls_score','gt_boxes_bv', 'gt_boxes_3d', 'im_info')
             .anchor_target_layer(_feat_stride[0], anchor_scales, name = 'rpn_data' ))

        # Loss of rpn_cls & rpn_boxes
        # anchor_num * xyzhlw
        # offset
        (self.feed('rpn_conv/3x3')
             .conv(1,1,len(anchor_scales)*2*6, 1, 1, padding='VALID', relu = False, name='rpn_bbox_pred'))

        #========= RoI Proposal ============
        # Lidar Bird View
        (self.feed('rpn_cls_score')
             .reshape_layer(2,name = 'rpn_cls_score_reshape')
             .softmax(name='rpn_cls_prob'))

        (self.feed('rpn_cls_prob')
             .reshape_layer(len(anchor_scales)*2*2,name = 'rpn_cls_prob_reshape'))

        (self.feed('rpn_cls_prob_reshape','rpn_bbox_pred','im_info', 'calib')
             .proposal_layer_3d(_feat_stride[0], 'TRAIN', name = 'rpn_rois'))

        (self.feed('rpn_rois', 'gt_boxes_bv', 'gt_boxes_3d', 'gt_boxes_corners', 'calib')
             .proposal_target_layer_3d(n_classes, name='roi_data_3d'))

        (self.feed('roi_data_3d')
             .proposal_transform(target='img', name='roi_data_img'))
        (self.feed('roi_data_3d')
             .proposal_transform(target='bv', name='roi_data_bv'))     

        (self.feed('conv5_3', 'roi_data_bv')
          .roi_pool(7, 7, 1.0/8, name='pool_5')
          .fc(2048, name='fc6_1')
          .dropout(self.keep_prob, name='drop6')
          .fc(2048, name='fc7_1')
          .dropout(self.keep_prob, name='drop7'))

        # image
        #(self.feed('deconv_2x_2', 'roi_data_img')
        (self.feed('conv5_3_2', 'roi_data_img')
             .roi_pool(7, 7, 1.0/8, name='pool_5_2')
             .fc(2048, name='fc6_2')
             .dropout(self.keep_prob, name='drop6_2')
              .fc(2048, name='fc7_2')
              .dropout(self.keep_prob, name='drop7_2'))

        # fusion
        (self.feed('drop7', 'drop7_2')
             .concat(axis=1, name='concat1')
             .dropout(self.keep_prob, name='drop7')
             .fc(n_classes, relu=False, name='cls_score')
             .softmax(name='cls_prob'))

        (self.feed('drop7')
             .fc(n_classes*24, relu=False, name='bbox_pred')) # (x0-x7,y0-y7,z0-z7)
```

In [10]:
print 'Use network `{:s}` in training'.format(network_name)

Use network `MV3D_train` in training


# 4. train_net()

In [16]:
pretrained_model = "/workspace/MV3D_TF/data/pretrain_model/VGG_imagenet.npy"
max_iters = 1
#train_net(network, imdb, roidb, output_dir, pretrained_model=pretrained_model, max_iters=max_iters)

```python
def train_net(network, imdb, roidb, output_dir, pretrained_model=None, max_iters=10000):
    """Train a Fast R-CNN network."""
    roidb = filter_roidb(roidb)
    saver = tf.train.Saver(max_to_keep=100)

    with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
        sw = SolverWrapper(sess, saver, network, imdb, roidb, output_dir, pretrained_model=pretrained_model)
        print 'Solving...'
        sw.train_model(sess, max_iters)
        print 'done solving'
```
## 4.2 filter_roidb(roidb) : 이미지가 없는 RoI들 제거

```python
def filter_roidb(roidb):
    """Remove roidb entries that have no usable RoIs."""
    
  return filtered_roidb
```

## 4.3 sw.train_model() : 메인!!!
```python
    def train_model(self, sess, max_iters):
        """Network training loop."""

        data_layer = get_data_layer(self.roidb, self.imdb.num_classes)   ## 4.3.1 입력층 중요 

        # RPN
        
        # bounding box regression L1 loss
        
        #  rpn_bbox_targets = tf.transpose(self.net.get_output('rpn_data')[1],[0,2,3,1])
        rpn_bbox_targets = self.net.get_output('rpn_data')[1]

        rpn_smooth_l1 = self._modified_smooth_l1(3.0, rpn_bbox_pred, rpn_bbox_targets)

        # R-CNN
        # classification loss
        cls_score = self.net.get_output('cls_score')

        # bounding box regression L1 loss
        bbox_pred = self.net.get_output('bbox_pred')
        bbox_targets = self.net.get_output('roi_data_3d')[3]

        smooth_l1 = self._modified_smooth_l1(3.0, bbox_pred, bbox_targets)

        # final loss
        loss = cross_entropy + loss_box + rpn_cross_entropy +  rpn_loss_box

        # optimizer and learning rate
        # global_step = tf.Variable(0, trainable=False)
        # lr = tf.train.exponential_decay(cfg.TRAIN.LEARNING_RATE, global_step,
        #                                 cfg.TRAIN.STEPSIZE, 0.1, staircase=True)
        #  momentum = cfg.TRAIN.MOMENTUM
        #  train_op = tf.train.MomentumOptimizer(lr, momentum).minimize(loss, global_step=global_step)
        lr = 0.00001
        # train_op = tf.train.GradientDescentOptimizer(lr).minimize(loss)
        train_op = tf.train.AdamOptimizer(lr).minimize(loss)


        # iintialize variables
        sess.run(tf.global_variables_initializer())
        self.net.load(self.pretrained_model, sess, self.saver, True)

        for iter in range(max_iters):
            # get one batch
            blobs = data_layer.forward()

            # Make one SGD update
            feed_dict={self.net.image_data: blobs['image_data'],
                       self.net.lidar_bv_data: blobs['lidar_bv_data'],
                       self.net.im_info: blobs['im_info'],
                       self.net.keep_prob: 0.5,
                       self.net.gt_boxes: blobs['gt_boxes'],
                       self.net.gt_boxes_bv: blobs['gt_boxes_bv'],
                       self.net.gt_boxes_3d: blobs['gt_boxes_3d'],
                       self.net.gt_boxes_corners: blobs['gt_boxes_corners'],
                       self.net.calib: blobs['calib']}


#########
rpn_bbox_pred_out,rpn_loss_cls_value, rpn_loss_box_value,loss_cls_value, loss_box_value, _ 
    = sess.run([rpn_bbox_pred, rpn_cross_entropy, rpn_loss_box, cross_entropy, loss_box, train_op],feed_dict=feed_dict, options=run_options, run_metadata=run_metadata)
##########


```
### 4.3.1 get_data_layer() : 입력층 

```python 
def get_data_layer(roidb, num_classes):
    """return a data layer."""
    if cfg.TRAIN.HAS_RPN:
        if cfg.IS_MULTISCALE:
            layer = GtDataLayer(roidb)
        else:
            layer = RoIDataLayer(roidb, num_classes)
    else:
        layer = RoIDataLayer(roidb, num_classes)
```

#### 4.3.2 RoIDataLayer
```python
class RoIDataLayer(object):
    """Fast R-CNN data layer used for training."""

    def __init__(self, roidb, num_classes):
        """Set the roidb to be used by this layer during training."""
        self._roidb = roidb
        self._num_classes = num_classes
        self._shuffle_roidb_inds()
```